In [ ]:
# utilities
import re
import numpy as np
import pandas as pd
# plotting
# import seaborn as sns
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.utils.class_weight import compute_class_weight
!pip install nltk
!pip install tensorflow
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

In [ ]:
csv_data=csv_data = pd.read_csv('extended_googleplaystore_user_reviews.csv',error_bad_lines=False)
print(csv_data.head())

                     App                                  Translated_Review  \
0  10 Best Foods for You  I like eat delicious food. That's I'm cooking ...   
1  10 Best Foods for You    This help eating healthy exercise regular basis   
2  10 Best Foods for You                                                NaN   
3  10 Best Foods for You         Works great especially going grocery store   
4  10 Best Foods for You                                       Best idea us   

   sentences_count  characters_count  spaces_count  count_words  \
0              2.0             122.0          20.0         22.0   
1              1.0              47.0           6.0          7.0   
2              NaN               NaN           NaN          NaN   
3              1.0              42.0           5.0          6.0   
4              1.0              12.0           2.0          3.0   

   duplicates_count  chars_excl_spaces_count  emoji_count  \
0               6.0                    102.0          0.0   


<ipython-input-3-e405d43d1683>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  csv_data=csv_data = pd.read_csv('extended_googleplaystore_user_reviews.csv',error_bad_lines=False)


In [ ]:
# Data Preparation
csv_data = pd.read_csv('extended_googleplaystore_user_reviews.csv',error_bad_lines=False)
csv_data = csv_data[csv_data['Translated_Review'].notna()]
p=csv_data['original_Sentiment_Polarity'].max()
q=csv_data['original_Sentiment_Polarity'].min()

print(p,q,len(csv_data))


1.0 -1.0 4884


<ipython-input-4-b9ab68f82e6c>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  csv_data = pd.read_csv('extended_googleplaystore_user_reviews.csv',error_bad_lines=False)


In [ ]:
class SentimentSatisfaction:
    def __init__(self,satisfaction_index):
        self.satisfaction_index = satisfaction_index
    def get_sentiment_satisfaction(self):
        if self.satisfaction_index <= -0.6:
            return 0
        elif self.satisfaction_index <= -0.2:
            return 1
        elif self.satisfaction_index <= 0.2:
            return 2
        elif self.satisfaction_index <= 0.6:
            return 3
        else:
            return 4

satisfaction_class = {
    "very_negative":0,
    "negative":1,
    "neutral":2,
    "positive":3,
    "very_positive":4
    }
csv_data['result'] = csv_data['original_Sentiment_Polarity'].apply(lambda x:SentimentSatisfaction(x).get_sentiment_satisfaction())

In [ ]:
data = csv_data.copy()
data = csv_data[['Translated_Review','result','original_Sentiment_Polarity']]

In [ ]:
#-------------------------------------------------------------------------------------------------------------
# Cleaning and Wrangling of Data
#-------------------------------------------------------------------------------------------------------------

# Remove URLs and mentions from text
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: re.sub(r'http\S+', '', x))
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: re.sub(r'@\S+', '', x))



# Remove non-alphabetic characters and convert to lowercase
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x.lower()))

# Tokenize text
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: nltk.word_tokenize(x))
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: [value for value in x if not re.match(r'^-?\d+\.?\d*$', value)])


# Remove stopwords
stop_words = stopwords.words('english')
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: [word for word in x if word not in stop_words])

# Lemmatize text
lemmatizer = WordNetLemmatizer()
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Join tokens back into strings
data.loc[:,'Translated_Review'] = data['Translated_Review'].apply(lambda x: ' '.join(x))
data.head()

In [ ]:

# For Machine Learning
m_data = data.copy()
review_data = m_data['Translated_Review']
result_data = m_data['result']

vectorizer = CountVectorizer()

# Fit and transform the data using the vectorizer object
c_vectorized_data = vectorizer.fit_transform(review_data)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(c_vectorized_data, result_data, test_size=0.2, random_state=42)



In [ ]:
# Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)
print('Logistic Regression Accuracy:', accuracy_score(y_test, lr_preds))

# Support Vector Machine model
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
print('SVM Accuracy:', accuracy_score(y_test, svm_preds))

# BiLSTM model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(review_data)
sequences = tokenizer.texts_to_sequences(review_data)
max_length = max([len(seq) for seq in sequences])
vocab_size = len(tokenizer.word_index) + 1
padded_sequences = pad_sequences(sequences, maxlen=max_length)
bi_model = Sequential()
bi_model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
bi_model.add(Bidirectional(LSTM(64, return_sequences=True)))
bi_model.add(Dense(64, activation='relu'))
bi_model.add(Dropout(0.5))
bi_model.add(Dense(5, activation='softmax'))
bi_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
bi_model.fit(padded_sequences, np.array(result_data), epochs=10, batch_size=32, validation_split=0.2)
bi_preds = bi_model.predict_classes(X_test)
print('BiLSTM Accuracy:', accuracy_score(y_test, bi_preds))

# Create the VotingClassifier
voting_clf = VotingClassifier(estimators=[('logreg', lr_model), ('svm', svm_model), ('bilstm', bi_model)], voting='soft')
voting_clf.fit(X_train, y_train)
voting_preds = voting_clf.predict(X_test)
voting_accuracy = accuracy_score(y_test, voting_preds)
print('Voting Classifier Accuracy:', voting_accuracy)

# Final voting_lg_bilstm prediction
voting_lg_bilstm = voting_clf.predict(c_vectorized_data)
voting_lg_bilstm_accuracy = accuracy_score(result_data, voting_lg_bilstm)
print('Voting LG BiLSTM Accuracy:', voting_lg_bilstm_accuracy)

# Create the VotingClassifier with BiLSTM and Hyperparameters
voting_clf_hp = VotingClassifier(estimators=[('bilstm', bi_model), ('voting_clf', voting_clf)], voting='hard')
voting_clf_hp.fit(X_train, y_train)
voting_lg_bilstm_hp = voting_clf_hp.predict(c_vectorized_data)
voting_lg_bilstm_hp_accuracy = accuracy_score(result_data, voting_lg_bilstm_hp)
print('Voting LG BiLSTM with Hyperparameters Accuracy:', voting_lg_bilstm_hp_accuracy)


In [ ]:
# Logistic Regression model
lr_model1 = LogisticRegression()

# Train logistic regression models
lr_model1.fit(X_train, y_train)

# Logistic Regression predictions
lr_preds1 = lr_model1.predict(X_test)
print('Accuracy',accuracy_score(y_test,lr_preds1))

In [ ]:
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
print('Accuracy',accuracy_score(y_test,svm_pred))

In [ ]:
voting_clf = VotingClassifier(estimators=[('logreg', lr_model1), ('svm', svm_model)], voting='soft')
voting_clf.fit(X_train, y_train)
voting_pred = voting_clf.predict(X_test)
accuracy = accuracy_score(y_test, voting_pred)
print("Accuracy:", accuracy)

In [ ]:
# Plot the accuracies
labels = ['Logistic Regression', 'Support Vector Machine','Voting LR and SVM']
accuracies = [accuracy_score(y_test,lr_preds1), accuracy_score(y_test,svm_pred),accuracy_score(y_test, voting_pred)]

plt.bar(labels, accuracies)
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.title('Comparison of Logistic Regression, SVM and Ensembled')
# Add accuracy points above each data point
for label, accuracy in zip(labels, accuracies):
    plt.annotate(f'{accuracy:.2f}', (label, accuracy), textcoords="offset points", xytext=(0, 10), ha='center')

plt.show()

In [ ]:
# Logistic Regression and Count Vectorizer
# Split the dataset into training and testing sets
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(c_vectorized_data, result_data, test_size=0.2,random_state=42)

# Train a machine learning model on the training set
lr_model_hp = LogisticRegression()

# For HyperParameter Tuning
param_grid={
    'warm_start': [True], 
    'solver': ['sag'], 
    'penalty': ['l2'], 
    'max_iter': [200], 
    'C': [206.913808111479]
}

# Perform random search with cross-validation
grid_search = GridSearchCV(lr_model_hp, param_grid=param_grid, cv=5)
grid_search.fit(X_train_lr, y_train_lr)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Use the best model to make predictions
y_pred_lr = best_model.predict(X_test_lr)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test_lr, y_pred_lr)

# For Plot
results = grid_search.cv_results_
params = results['params']
mean_test_scores = results['mean_test_score']
# Evaluate the accuracy of the model
# Plot the accuracy scores
plt.figure(figsize=(12, 6))
plt.bar(range(len(params)), mean_test_scores, align='center', alpha=0.8)
plt.xticks(range(len(params)), [str(p) for p in params], rotation='vertical')
plt.xlabel('Hyperparameters')
plt.ylabel('Mean Test Score')
plt.title('GridSearchCV Results - Logistic Regression')
plt.tight_layout()
plt.show()
print("Accuracy:", accuracy)

In [ ]:
voting_clf_hp = VotingClassifier(estimators=[('grid_lr', grid_search), ('svm', svm_model)], voting='hard')
voting_clf_hp.fit(X_train, y_train)
voting_pred_hp = voting_clf.predict(X_test)
print("Accuracy with HP:", accuracy_score(y_test, voting_pred_hp))

In [ ]:
# Plot the accuracies
labels = ['Logistic Regression ', 'Ensemble LR HP and SVM']
accuracies = [accuracy, accuracy_score(y_test, voting_pred_hp)]

plt.bar(labels, accuracies)
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.title('Comparison of Logistic Regression HP, SVM and Ensembled')
# Add accuracy points above each data point
for label, accuracy in zip(labels, accuracies):
    plt.annotate(f'{accuracy:.2f}', (label, accuracy), textcoords="offset points", xytext=(0, 10), ha='center')

plt.show()

In [ ]:
# For Deep Learning
d_data = data.copy()
d_data = d_data[['Translated_Review','result']]
texts = d_data['Translated_Review']
sentiment = d_data['result']

# Tokenize the texts
tokenizer = Tokenizer(num_words=17599)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to have the same length
padded_sequences = pad_sequences(sequences, maxlen=300)
max_len = max(len(sequence) for sequence in sequences)


In [ ]:
X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(padded_sequences, sentiment, test_size=0.2, random_state=42)

# Define the BiLSTM model
bi_model = Sequential()
bi_model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_len))
bi_model.add(Bidirectional(LSTM(64, return_sequences=True)))
bi_model.add(Bidirectional(LSTM(64)))
bi_model.add(Dense(64, activation='relu'))
bi_model.add(Dense(5, activation='softmax'))  # 5 classes for fuzzy sentiment analysis

# Compile the bi_model
bi_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the bi_model
history = bi_model.fit(X_train_bi, y_train_bi, validation_data=(X_test_bi, y_test_bi), epochs=10, batch_size=32)

accuracy = history.history.get('accuracy')
val_acc = history.history.get('val_accuracy')
loss = history.history.get('loss')
val_loss = history.history.get('val_loss')

epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
# For Polarity Score Estimation
p_data = data.copy()
sia = SentimentIntensityAnalyzer()
sentiment_scores = []
for review in p_data['Translated_Review']:
    scores = sia.polarity_scores(review)
    sentiment_scores.append(scores['compound'])
p_data['sentiment_score'] = sentiment_scores
# p_data['predicted_label'] = p_data['sentiment_score'].apply(lambda x:SentimentSatisfaction(x).get_sentiment_satisfaction())
p_data['predicted_label'] = np.argmax(p_data['sentiment_score'])
accuracy = accuracy_score(p_data['result'], p_data['predicted_label'])
print(p_data)


In [ ]:
voting_lg_bilstm = VotingClassifier(estimators=[('bilstm', bi_model), ('voting_clf_hp', voting_clf_hp)], voting='soft')
voting_lg_bilstm.fit(X_train_bi, y_train_bi)
voting_pred = voting_lg_bilstm.predict(X_test_bi)
accuracy = accuracy_score(y_test, voting_pred)
print("Accuracy:", accuracy)

In [ ]:
# Graph Plot

In [ ]:
# Custom transformer to extract sentiment scores using SentimentIntensityAnalyzer
class SentimentIntensityTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.sid = SentimentIntensityAnalyzer()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        scores = []
        for text in X:
            sentiment_scores = self.sid.polarity_scores(text)
            scores.append(list(sentiment_scores.values()))
        return scores

In [ ]:
# Instantiate the models
sid_transformer = SentimentIntensityTransformer()

# Define the voting classifier
voting_final = VotingClassifier(estimators=[
    ('sid', sid_transformer),
    ('voting_lg_bilstm', voting_lg_bilstm)
], voting='soft')

voting_pred_final = voting_lg_bilstm.predict(X_test_bi)
accuracy = accuracy_score(y_test, voting_pred_final)
print("Accuracy:", accuracy)

In [ ]:
# Graph Plot